In [1]:
import os
os.chdir("/home/wmbio/WORK/gitworking/Multi-omics-intergration")
os.getcwd()

'/home/wmbio/WORK/gitworking/Multi-omics-intergration'

## **Module**

In [2]:
from wmbio import * 

/home/wmbio/anaconda3/envs/multiomics-cpu/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


## **Best Sub-Group Selection**

* **FILE PATH**

In [3]:
# PATH
CANCER_TYPE = "BRCA"
GROUP_PHTH = os.getcwd() + '/group/'
PNG_PATH = os.getcwd() + '/png/'
GROUP_VALIDATION_PATH = os.getcwd() + '/group_validation/'
DEG_PATH = os.getcwd() + "/best_deg/"
RDATA_PATH = os.getcwd() + "/RAW_DATA/GDC_PREPARE/"
RAW_PATH = os.getcwd() + "/RAW_DATA/"

METHOD = 'deseq2'
LOGFC=1
FDR=0.05

In [7]:
# Load Validation score
col=['FILENAME','Log Rank Test','Silhouette','RNA_ANOVA_F1','RNA_RF_F1',
     'miRNA_ANOVA_F1','miRNA_RF_F1','Methylation_ANOVA_F1','Methylation_RF_F1']

group_score = pd.read_csv(GROUP_VALIDATION_PATH + 'BRCA_validation.csv', usecols=col)

# Q3 value
SILHOUETTE = group_score.Silhouette.quantile(.7)
RNA_ANOVA = group_score.RNA_ANOVA_F1.quantile(.7)
RNA_RF = group_score.RNA_RF_F1.quantile(.7)
MIRNA_ANOVAR = group_score.miRNA_ANOVA_F1.quantile(.7)
MIRNA_RF = group_score.miRNA_RF_F1.quantile(.7)
MT_ANOVAR = group_score.Methylation_ANOVA_F1.quantile(.7)
MT_RF = group_score.Methylation_RF_F1.quantile(.7)

print("SILHOUETTE Q3 : ", SILHOUETTE)
print("RNA_ANOVA Q3 : ", RNA_ANOVA)
print("RNA_RF Q3 : ", RNA_RF)
print("MIRNA_ANOVAR Q3 : ", MIRNA_ANOVAR)
print("MIRNA_RF Q3 : ", MIRNA_RF)
print("MT_ANOVAR Q3 : ", MT_ANOVAR)
print("MT_RF Q3 : ", MT_RF)

# Condition for Filtering
filter_cond = (group_score['Silhouette'] >= SILHOUETTE) & (group_score['Log Rank Test'] < 0.05) & \
              ((group_score['RNA_ANOVA_F1'] > RNA_ANOVA) | (group_score['RNA_RF_F1'] > RNA_RF)) & \
              ((group_score['miRNA_ANOVA_F1'] > MIRNA_ANOVAR) | (group_score['miRNA_RF_F1'] > MIRNA_RF)) & \
              ((group_score['Methylation_ANOVA_F1'] > MT_ANOVAR) | (group_score['Methylation_RF_F1'] > MT_RF))

SILHOUETTE Q3 :  0.2395
RNA_ANOVA Q3 :  89.47197539242242
RNA_RF Q3 :  89.93464052287582
MIRNA_ANOVAR Q3 :  83.20727956343404
MIRNA_RF Q3 :  84.31372549
MT_ANOVAR Q3 :  91.234449758
MT_RF Q3 :  91.83006535947712


In [ ]:
group_score = group_score[filter_cond].sort_values(["Silhouette"], ascending = (False))

In [ ]:
bestSubgroup = group_score.FILENAME.to_list()

In [ ]:
if len(bestSubgroup) > 100:
    random.seed(331)
    bestSubgroup = random.sample(bestSubgroup, k=100)

In [ ]:
DEG_CHECK = "_".join([CANCER_TYPE, METHOD.upper(), bestSubgroup[0]]) + ".txt"

In [ ]:
# DEA
dea_result = list()
for best_group in bestSubgroup:
    
    DEG_CHECK = "_".join([CANCER_TYPE, METHOD.upper(), best_group]) + ".txt"
    SAMPLE_GROUP = GROUP_PHTH + CANCER_TYPE + "/" + CANCER_TYPE + "_GROUP_" + best_group + ".txt"
    
    if os.path.isfile(DEG_PATH + CANCER_TYPE + "/" + DEG_CHECK):
        deg_list = pd.read_csv(DEG_PATH + CANCER_TYPE + "/" + DEG_CHECK, sep = "\t")
    else :
        # DEG Extraction
        deg_list = deg_extract(log_fc=LOGFC, fdr=FDR,
                       cancer_type=CANCER_TYPE, 
                       sample_group=SAMPLE_GROUP, deg_path=DEG_PATH, 
                       file_name=best_group,
                       rdata_path=RDATA_PATH,
                       method=METHOD,
                       batch_removal=True,
                       raw_path = RAW_PATH)
    
    dea_result.append(deg_list)

In [ ]:
# Filter DEA
# combine result
if METHOD == 'all':
    dea_combine = list(map(deseq2_edger_combine, dea_result))
    dea_combine = [col_rename(dea_combine[index], index, bestSubgroup) for index in range(len(dea_combine))]
    dea_combine = reduce(lambda left, right : pd.merge(left, right, left_on='gene', right_on='gene', how = 'outer'), dea_combine)
elif METHOD == 'deseq2' :
    dea_combine = list(map(lambda d : d[["row", "log2FoldChange"]], dea_result))
    dea_combine = [col_rename(dea_combine[index], index, bestSubgroup) for index in range(len(dea_combine))]
    dea_combine = reduce(lambda left, right : pd.merge(left, right, left_on='gene', right_on='gene', how = 'inner'), dea_combine)

# median & mean
dea_combine["SubGroup-log2FC_median"] = dea_combine.iloc[:, 1:].median(axis=1)
dea_combine["SubGroup-log2FC_mean"] = dea_combine.iloc[:, 1:].mean(axis=1)

* **NT vs TP DEA(Deseq2)**

In [4]:
nt_tp_deseq2 = deg_extract_normal(log_fc=0, pvalue=0.1, cancer_type=CANCER_TYPE, 
                                  rdata_path=RDATA_PATH, deg_path=DEG_PATH, batch_removal=True)

nt_tp_deseq2_col = nt_tp_deseq2[['row', 'log2FoldChange', 'pvalue']]
nt_tp_deseq2_col.columns = ['gene', 'NT-TP_log2FoldChange', 'pvalue']

Number of significant surrogate variables is:  2 
Iteration (out of 5 ):1  2  3  4  5  

In [5]:
nt_tp_deseq2

,row,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
1,A1BG,491.160929,0.203242,0.104740,1.940448,5.232527e-02,0.251120
2,A2M,43832.644040,0.167815,0.076730,2.187092,2.873582e-02,0.186639
15,ABP1,206.737300,-1.022170,0.235060,-4.348553,1.370386e-05,0.001334
19,ACACA,6917.202891,-0.418246,0.081624,-5.124074,2.990029e-07,0.000055
20,ACACB,3255.246327,0.322257,0.129071,2.496740,1.253409e-02,0.121372
...,...,...,...,...,...,...,...
14862,LOC100131691,89.162272,0.153343,0.089775,1.708081,8.762138e-02,0.320941
14867,LOC100132288,624.076559,0.243165,0.100096,2.429318,1.512726e-02,0.134158
14869,GAGE12D,185.618664,-2.723621,1.387746,-1.962622,4.969009e-02,0.244640
14876,C17orf96,534.111196,-0.211488,0.122776,-1.722560,8.496813e-02,0.316389


In [5]:
nt_tp_deseq2

,row,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
1,A1BG,491.160929,-0.203242,0.104740,-1.940448,5.232527e-02,0.251120
2,A2M,43832.644040,-0.167815,0.076730,-2.187092,2.873582e-02,0.186639
15,ABP1,206.737300,1.022170,0.235060,4.348553,1.370386e-05,0.001334
19,ACACA,6917.202891,0.418246,0.081624,5.124074,2.990029e-07,0.000055
20,ACACB,3255.246327,-0.322257,0.129071,-2.496740,1.253409e-02,0.121372
...,...,...,...,...,...,...,...
14862,LOC100131691,89.162272,-0.153343,0.089775,-1.708081,8.762138e-02,0.320941
14867,LOC100132288,624.076559,-0.243165,0.100096,-2.429318,1.512726e-02,0.134158
14869,GAGE12D,185.618664,2.723621,1.387746,1.962622,4.969009e-02,0.244640
14876,C17orf96,534.111196,0.211488,0.122776,1.722560,8.496813e-02,0.316389


In [6]:
nt_tp_deseq2

,row,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
1,A1BG,491.160929,0.203242,0.104740,1.940448,5.232527e-02,0.251120
2,A2M,43832.644040,0.167815,0.076730,2.187092,2.873582e-02,0.186639
15,ABP1,206.737300,-1.022170,0.235060,-4.348553,1.370386e-05,0.001334
19,ACACA,6917.202891,-0.418246,0.081624,-5.124074,2.990029e-07,0.000055
20,ACACB,3255.246327,0.322257,0.129071,2.496740,1.253409e-02,0.121372
...,...,...,...,...,...,...,...
14862,LOC100131691,89.162272,0.153343,0.089775,1.708081,8.762138e-02,0.320941
14867,LOC100132288,624.076559,0.243165,0.100096,2.429318,1.512726e-02,0.134158
14869,GAGE12D,185.618664,-2.723621,1.387746,-1.962622,4.969009e-02,0.244640
14876,C17orf96,534.111196,-0.211488,0.122776,-1.722560,8.496813e-02,0.316389


* **Write Result**

In [ ]:
pd.merge(left=dea_combine, right=nt_tp_deseq2_col, left_on='gene', right_on='gene', how = 'left').to_csv('temp.csv', index = False)